In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/vv/sc6b0f596n7dy9mbxlcnn1180000gq/T/ipykernel_70624/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_100M_whitespace"

#model = GPT2LMHeadModel.from_pretrained(model_path)
#tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
#lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")
lm_scorer = scorer.IncrementalLMScorer('gpt2', device = "cpu")

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 8.975661277770996),
 ('that', 3.035094738006592),
 ('your', 7.0676116943359375),
 ('friend', 7.013303756713867),
 ('gave', 8.271688461303711),
 ('a', 4.564513206481934),
 ('baguette', 19.14437147974968),
 ('to', 1.5210200548171997),
 ('Mary', 11.295434951782227),
 ('last', 9.892695426940918),
 ('weekend', 5.39995813369751),
 ('.', 1.7855156660079956)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

<>:24: SyntaxWarning: invalid escape sequence '\w'
<>:24: SyntaxWarning: invalid escape sequence '\w'
/var/folders/vv/sc6b0f596n7dy9mbxlcnn1180000gq/T/ipykernel_70624/2872888706.py:24: SyntaxWarning: invalid escape sequence '\w'
  sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)


In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part) for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [8]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

FileNotFoundError: [Errno 2] No such file or directory: 'test_sentences/Gap Distance.csv'

In [ ]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [ ]:
# modifier: short_mod, med_mod, long_mod

In [ ]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 25.3498
Min. group size: 16
Log-Likelihood: -979.6989
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,19.116,1.016,18.815,0.000,17.125,21.107
1,gap_distance[T.short],-2.096,0.796,-2.632,0.008,-3.656,-0.535
2,gap_distance[T.medium],-1.859,0.796,-2.336,0.020,-3.420,-0.299
3,gap_distance[T.long],-2.025,0.796,-2.543,0.011,-3.585,-0.464
4,wh_numeric,0.472,0.563,0.838,0.402,-0.632,1.575
5,wh_numeric:gap_distance[T.short],-0.463,0.796,-0.582,0.561,-2.023,1.097
6,wh_numeric:gap_distance[T.medium],-0.477,0.796,-0.600,0.549,-2.038,1.083
7,wh_numeric:gap_distance[T.long],-0.596,0.796,-0.749,0.454,-2.156,0.964
8,gap_numeric,-1.563,0.563,-2.777,0.005,-2.667,-0.460
9,gap_numeric:gap_distance[T.short],0.737,0.796,0.926,0.354,-0.823,2.298



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,14.308,1.058


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 25.2259
Min. group size: 16
Log-Likelihood: -1004.6053
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,18.300,0.954,19.183,0.000,16.430,20.169
1,wh_numeric,0.321,0.447,0.720,0.472,-0.554,1.197
2,gap_numeric,-1.557,0.447,-3.487,0.000,-2.433,-0.682
3,wh_numeric:gap_numeric,-2.786,0.447,-6.239,0.000,-3.661,-1.911
4,gap_distance_length,-0.019,0.010,-1.939,0.052,-0.039,0.000
5,wh_numeric:gap_distance_length,-0.007,0.010,-0.674,0.501,-0.026,0.013
6,gap_numeric:gap_distance_length,0.013,0.010,1.305,0.192,-0.006,0.032
7,wh_numeric:gap_numeric:gap_distance_length,0.008,0.010,0.791,0.429,-0.011,0.027



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,14.175,1.050


Gap Postion == PP/Goal

In [ ]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 40.7864
Min. group size: 16
Log-Likelihood: -1044.3291
Max. group size: 16
Converged: No
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,17.916,0.976,18.348,0.000,16.002,19.829
1,gap_distance[T.short],-2.630,1.010,-2.605,0.009,-4.610,-0.651
2,gap_distance[T.medium],-1.435,1.010,-1.421,0.155,-3.414,0.544
3,gap_distance[T.long],-1.608,1.010,-1.592,0.111,-3.587,0.371
4,wh_numeric,-0.720,0.714,-1.009,0.313,-2.120,0.679
5,wh_numeric:gap_distance[T.short],0.237,1.010,0.235,0.814,-1.742,2.216
6,wh_numeric:gap_distance[T.medium],-0.016,1.010,-0.016,0.987,-1.996,1.963
7,wh_numeric:gap_distance[T.long],0.038,1.010,0.038,0.970,-1.941,2.017
8,gap_numeric,5.154,0.714,7.218,0.000,3.754,6.553
9,gap_numeric:gap_distance[T.short],-0.739,1.010,-0.731,0.464,-2.718,1.240



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,8.871,0.853


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 40.9358
Min. group size: 16
Log-Likelihood: -1072.7113
Max. group size: 16
Converged: No
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,16.826,0.885,19.009,0.000,15.091,18.561
1,wh_numeric,-0.621,0.569,-1.092,0.275,-1.736,0.494
2,gap_numeric,4.904,0.569,8.621,0.000,3.790,6.019
3,wh_numeric:gap_numeric,-1.939,0.569,-3.408,0.001,-3.054,-0.824
4,gap_distance_length,-0.009,0.013,-0.739,0.460,-0.034,0.015
5,wh_numeric:gap_distance_length,-0.001,0.012,-0.078,0.938,-0.025,0.024
6,gap_numeric:gap_distance_length,0.014,0.012,1.124,0.261,-0.010,0.039
7,wh_numeric:gap_numeric:gap_distance_length,0.014,0.012,1.156,0.248,-0.010,0.039



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,9.147,0.899


Embedded Clause

In [ ]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [ ]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2471
Min. group size: 16
Log-Likelihood: -282.3213
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.749,0.133,50.816,0.000,6.489,7.009
1,gap_distance[T.short],0.363,0.079,4.613,0.000,0.209,0.517
2,gap_distance[T.medium],-0.139,0.079,-1.771,0.076,-0.293,0.015
3,gap_distance[T.long],0.152,0.079,1.940,0.052,-0.002,0.307
4,wh_numeric,-0.021,0.056,-0.383,0.701,-0.130,0.088
5,wh_numeric:gap_distance[T.short],0.013,0.079,0.164,0.870,-0.141,0.167
6,wh_numeric:gap_distance[T.medium],0.060,0.079,0.761,0.447,-0.094,0.214
7,wh_numeric:gap_distance[T.long],0.052,0.079,0.659,0.510,-0.102,0.206
8,gap_numeric,0.209,0.056,3.763,0.000,0.100,0.318
9,gap_numeric:gap_distance[T.short],0.009,0.079,0.119,0.906,-0.145,0.163



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.291,0.207


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2786
Min. group size: 16
Log-Likelihood: -307.0767
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.866,0.129,53.223,0.000,6.614,7.119
1,wh_numeric,-0.019,0.047,-0.407,0.684,-0.111,0.073
2,gap_numeric,0.222,0.047,4.727,0.000,0.130,0.314
3,wh_numeric:gap_numeric,-0.350,0.047,-7.451,0.000,-0.442,-0.258
4,gap_distance_length,-0.001,0.001,-0.634,0.526,-0.003,0.001
5,wh_numeric:gap_distance_length,0.001,0.001,0.792,0.428,-0.001,0.003
6,gap_numeric:gap_distance_length,-0.001,0.001,-1.261,0.207,-0.003,0.001
7,wh_numeric:gap_numeric:gap_distance_length,0.003,0.001,2.778,0.005,0.001,0.005



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.288,0.194


Embedded Clause - PP/Goal Gap Position

In [ ]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2350
Min. group size: 16
Log-Likelihood: -274.1055
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.855,0.126,54.305,0.000,6.608,7.102
1,gap_distance[T.short],0.312,0.077,4.066,0.000,0.161,0.462
2,gap_distance[T.medium],-0.201,0.077,-2.618,0.009,-0.351,-0.050
3,gap_distance[T.long],0.081,0.077,1.058,0.290,-0.069,0.231
4,wh_numeric,0.082,0.054,1.506,0.132,-0.025,0.188
5,wh_numeric:gap_distance[T.short],0.011,0.077,0.144,0.885,-0.139,0.161
6,wh_numeric:gap_distance[T.medium],-0.009,0.077,-0.115,0.909,-0.159,0.141
7,wh_numeric:gap_distance[T.long],-0.016,0.077,-0.205,0.838,-0.166,0.135
8,gap_numeric,0.387,0.054,7.142,0.000,0.281,0.493
9,gap_numeric:gap_distance[T.short],-0.063,0.077,-0.824,0.410,-0.213,0.087



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.260,0.190


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2641
Min. group size: 16
Log-Likelihood: -298.1617
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.958,0.122,56.894,0.000,6.718,7.198
1,wh_numeric,0.091,0.046,1.985,0.047,0.001,0.180
2,gap_numeric,0.376,0.046,8.234,0.000,0.287,0.466
3,wh_numeric:gap_numeric,-0.173,0.046,-3.784,0.000,-0.262,-0.083
4,gap_distance_length,-0.002,0.001,-1.536,0.124,-0.004,0.000
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.350,0.726,-0.002,0.002
6,gap_numeric:gap_distance_length,-0.002,0.001,-1.834,0.067,-0.004,0.000
7,wh_numeric:gap_numeric:gap_distance_length,0.002,0.001,1.926,0.054,-0.000,0.004



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.257,0.178


#### Double Gaps

In [ ]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [ ]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']

In [ ]:
sentence_df['gap_count'] = pd.Categorical(sentence_df['gap_count'], categories = [1, 0, 2], ordered = True)

Embedded Clause

In [ ]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_count']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3119
Min. group size: 8
Log-Likelihood: -172.6716
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.245,0.243,29.783,0.000,6.768,7.722
1,gap_count[T.0],-0.727,0.108,-6.723,0.000,-0.939,-0.515
2,gap_count[T.2],1.463,0.108,13.525,0.000,1.251,1.675
3,wh_numeric,-0.088,0.062,-1.409,0.159,-0.210,0.034
4,wh_numeric:gap_count[T.0],0.536,0.108,4.956,0.000,0.324,0.748
5,wh_numeric:gap_count[T.2],0.064,0.108,0.593,0.553,-0.148,0.276



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.105,0.710


#### Wh-Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 19.7700
Min. group size: 12
Log-Likelihood: -710.6229
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,16.954,0.942,18.007,0.000,15.109,18.800
1,comp_type[T.that],-0.252,0.703,-0.358,0.720,-1.630,1.126
2,comp_type[T.whether],-0.095,0.703,-0.135,0.893,-1.473,1.283
3,wh_numeric,-0.084,0.497,-0.169,0.866,-1.058,0.890
4,wh_numeric:comp_type[T.that],-0.348,0.703,-0.495,0.620,-1.726,1.030
5,wh_numeric:comp_type[T.whether],-0.046,0.703,-0.065,0.948,-1.424,1.332
6,gap_numeric,2.763,0.497,5.557,0.000,1.788,3.737
7,gap_numeric:comp_type[T.that],0.191,0.703,0.271,0.786,-1.187,1.569
8,gap_numeric:comp_type[T.whether],2.018,0.703,2.870,0.004,0.640,3.396
9,wh_numeric:gap_numeric,-2.320,0.497,-4.667,0.000,-3.294,-1.346



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,12.786,1.100


Embedded Clause

In [ ]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0841
Min. group size: 12
Log-Likelihood: -112.2177
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.506,0.197,38.019,0.000,7.119,7.893
1,comp_type[T.that],-0.464,0.046,-10.110,0.000,-0.554,-0.374
2,comp_type[T.whether],0.459,0.046,10.007,0.000,0.369,0.549
3,wh_numeric,0.013,0.032,0.398,0.691,-0.051,0.076
4,wh_numeric:comp_type[T.that],0.023,0.046,0.503,0.615,-0.067,0.113
5,wh_numeric:comp_type[T.whether],-0.043,0.046,-0.948,0.343,-0.133,0.046
6,gap_numeric,0.287,0.032,8.847,0.000,0.223,0.350
7,gap_numeric:comp_type[T.that],-0.010,0.046,-0.214,0.831,-0.100,0.080
8,gap_numeric:comp_type[T.whether],0.230,0.046,5.022,0.000,0.140,0.320
9,wh_numeric:gap_numeric,-0.242,0.032,-7.475,0.000,-0.306,-0.179



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.759,0.894


#### Adjunct Islands 

In [ ]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 18.2275
Min. group size: 12
Log-Likelihood: -705.1112
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.216,1.067,20.815,0.000,20.125,24.308
1,adjunct_type[T.back],-1.280,0.675,-1.896,0.058,-2.603,0.043
2,adjunct_type[T.front],-0.935,0.675,-1.385,0.166,-2.258,0.388
3,wh_numeric,0.619,0.477,1.297,0.195,-0.316,1.555
4,wh_numeric:adjunct_type[T.back],-0.689,0.675,-1.021,0.307,-2.012,0.634
5,wh_numeric:adjunct_type[T.front],-0.639,0.675,-0.947,0.344,-1.962,0.684
6,gap_numeric,3.805,0.477,7.972,0.000,2.870,4.741
7,gap_numeric:adjunct_type[T.back],2.798,0.675,4.145,0.000,1.475,4.121
8,gap_numeric:adjunct_type[T.front],1.992,0.675,2.951,0.003,0.669,3.315
9,wh_numeric:gap_numeric,-1.424,0.477,-2.984,0.003,-2.360,-0.489



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,18.227,1.567


Embedded Clause

In [ ]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0867
Min. group size: 12
Log-Likelihood: -110.3325
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.550,0.153,42.789,0.000,6.250,6.850
1,adjunct_type[T.back],0.021,0.047,0.452,0.651,-0.070,0.112
2,adjunct_type[T.front],0.515,0.047,11.055,0.000,0.423,0.606
3,wh_numeric,0.182,0.033,5.517,0.000,0.117,0.246
4,wh_numeric:adjunct_type[T.back],0.015,0.047,0.319,0.749,-0.076,0.106
5,wh_numeric:adjunct_type[T.front],0.042,0.047,0.905,0.365,-0.049,0.133
6,gap_numeric,0.099,0.033,3.014,0.003,0.035,0.164
7,gap_numeric:adjunct_type[T.back],0.130,0.047,2.789,0.005,0.039,0.221
8,gap_numeric:adjunct_type[T.front],0.134,0.047,2.881,0.004,0.043,0.225
9,wh_numeric:gap_numeric,-0.160,0.033,-4.847,0.000,-0.224,-0.095



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.447,0.523


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')